In [1]:
from numba import cuda, jit, float32
import numpy as np
import matplotlib.pyplot as plt
from time import time

In [2]:
@jit(nopython=True)
def inner_product(x, y):
    out = 0
    
    for i in range(x.size):
        out += x[i] * y[i]

    return out

In [3]:
@jit(nopython=True)
def inner_product_for_grad(x, y, b):
    out = b * (-1)
    
    for i in range(x.size):
        out += x[i] * y[i]
    

    return out

In [4]:
@cuda.jit
def matrix_vector_multiplication(A, x, b, out):
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x

    i = bx * TPB + tx

    if i < out.shape[0]:
        out[i] = inner_product_for_grad(A[i,:], x, b[i])

In [5]:
TPB = 16

@cuda.jit
def get_grad(A, x, b, out):
    sA = cuda.shared.array(shape=(TPB,TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB), dtype=float32)

    i = cuda.grid(1)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y

    if i >= out.shape[0]:
        ## Quit if (x) is outside of valid out boundary
        return

    tmp = 0.
    for j in range(int(A.shape[0] / TPB)):
        ## Preload data into shared memory
        sA[tx, ty] = A.T[i, ty + j * TPB]
        sB[tx] = inner_product_for_grad(A[tx + j * TPB,:], x, b[tx + j * TPB])

        ## Wait until all threads finish proloading
        cuda.syncthreads()

        ## Computes partial product on the shared memory
        for k in range(TPB):
            tmp += sA[tx, k] * sB[tx]

        ## Wait until all threads finish computing
        cuda.syncthreads()

    out[i] = tmp

In [6]:
A = np.random.rand(1000,1000)
b = np.random.rand(1000)
x = np.random.rand(1000)
out = np.zeros((1000))

In [8]:
A_ = cuda.to_device(A)
b_ = cuda.to_device(b)
x_ = cuda.to_device(x)
out_ = cuda.to_device(out)

In [9]:
## Configure the blocks
threadsperblock = (TPB,TPB)
blockspergrid_x = int(np.ceil(A.shape[0] / threadsperblock[1]))
blockspergrid_y = int(np.ceil(A.shape[1] / threadsperblock[0]))
blockspergrid = (blockspergrid_x, blockspergrid_y)
get_grad[blockspergrid, threadsperblock](A_, x_, b_, out_)

In [7]:
## In CPU with numpy
%%time 
for i in range(500):
    grad = A.T @ (A @ x - b)
## almost 14.6 ms takes to calculate 

CPU times: user 520 ms, sys: 23.5 ms, total: 543 ms
Wall time: 283 ms


In [12]:
## Start the kernel
%%time 
for i in range(500):
    get_grad[blockspergrid, threadsperblock](A_, x_, b_, out_)
## almost 1.25 ms takes to calculate

CPU times: user 164 ms, sys: 4.06 ms, total: 168 ms
Wall time: 171 ms


In [10]:
grad[:10]

array([119761.04680633, 121216.75281535, 122725.00075552, 122477.03220112,
       125388.58226385, 126088.68376316, 123372.65253855, 121088.90936018,
       122421.62081622, 125534.37401711])

In [11]:
out = out_.copy_to_host()
out[:10]

array([119125.63091317, 120502.6331888 , 121845.22478592, 121861.09284684,
       123786.02289252, 124991.88896155, 121793.29539299, 119452.03149002,
       121057.25938263, 124614.84846944])

2022_04_26.ipynb 에서 사용한 이상한... numba 사용말고 조금 더 개선된 방법(__shared memory__를 사용하는 방법)으로 gradient를 계산하는 것을 구현했습니다... 속도는 좀더 개선되었습니다...<br>
조금 걱정되는 것은 초반에 cuda.synchronize()로 thread들을 정렬시키는데에 시간이 꽤 오래걸려 실제 사용중에도 시간이 오래걸리는 원인이 되지 않을까 걱정됩니다... 오히려 없는것이 더 빠른 요상한 경우가 있네요...<br>
> @jit 데코레이터는 첫번째 컴파일시 코드를 조금 변형해 더 빠르게 만들어 주므로, 첫번째 컴파일은 굉장히(상대적으로) 오래걸립니다. 마치 C언어를 컴파일하는 느낌으로 이해하면 잘 와다았습니다.<br>

그리고 또, 실질적으로 학습시에 CPU, GPU간 통신의 latency 로 여기서 얻은 시간 단축이 무용지물이 될 가능성도 염두해두고 있습니다...<br>
마지막으로 오차가 0.0361이나 되는 것이 걱정됩니다... 계산상 과정은 똑같은데 세분화되면서 오차가 누적되었나 싶기도 하네요... 오차 계산시에 1e-6을 곱해준 것은 실질적인 learning rate가 $lr = \frac{1e-3}{A.shape[0]}$으로 계산되어서 $A.shape[0] = 1000$을 대입해준 $lr = 1e-6$입니다. 이러니 오차가 굉장히 크네요.